In [32]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from sqlalchemy import create_engine
from fuzzywuzzy import fuzz


/Users/aracelyvivanco/opt/anaconda3/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# Extract and Transform

### Extract FIFA Player attributes from CSV and load into DataFrame

In [8]:
players_file = "resources/players_20.csv"
players_df = pd.read_csv(players_file)
players_df.head()

,sofifa_id,player_url,short_name,long_name,age,dob,height_cm,weight_kg,nationality,club,...,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,32,1987-06-24,170,72,Argentina,FC Barcelona,...,68+2,66+2,66+2,66+2,68+2,63+2,52+2,52+2,52+2,63+2
1,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,34,1985-02-05,187,83,Portugal,Juventus,...,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3
2,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Junior,27,1992-02-05,175,68,Brazil,Paris Saint-Germain,...,66+3,61+3,61+3,61+3,66+3,61+3,46+3,46+3,46+3,61+3
3,200389,https://sofifa.com/player/200389/jan-oblak/20/...,J. Oblak,Jan Oblak,26,1993-01-07,188,87,Slovenia,Atlético Madrid,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,183277,https://sofifa.com/player/183277/eden-hazard/2...,E. Hazard,Eden Hazard,28,1991-01-07,175,74,Belgium,Real Madrid,...,66+3,63+3,63+3,63+3,66+3,61+3,49+3,49+3,49+3,61+3


### Select columns for Player information tables

In [9]:
# Select columns for DataFrame to import into Players SQL Table
players_cols = ["sofifa_id", "player_url", "short_name", "long_name", "age", "dob", "height_cm", "weight_kg", "nationality", "club",
"overall", "potential", "value_eur", "wage_eur", "player_positions", "preferred_foot"]

players_transformed= players_df[players_cols].copy()

players_transformed.head()

,sofifa_id,player_url,short_name,long_name,age,dob,height_cm,weight_kg,nationality,club,overall,potential,value_eur,wage_eur,player_positions,preferred_foot
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,32,1987-06-24,170,72,Argentina,FC Barcelona,94,94,95500000,565000,"RW, CF, ST",Left
1,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,34,1985-02-05,187,83,Portugal,Juventus,93,93,58500000,405000,"ST, LW",Right
2,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Junior,27,1992-02-05,175,68,Brazil,Paris Saint-Germain,92,92,105500000,290000,"LW, CAM",Right
3,200389,https://sofifa.com/player/200389/jan-oblak/20/...,J. Oblak,Jan Oblak,26,1993-01-07,188,87,Slovenia,Atlético Madrid,91,93,77500000,125000,GK,Right
4,183277,https://sofifa.com/player/183277/eden-hazard/2...,E. Hazard,Eden Hazard,28,1991-01-07,175,74,Belgium,Real Madrid,91,91,90000000,470000,"LW, CF",Right


In [10]:
# Select columns for DataFrame to import into Player_skill SQL Table
player_skill_cols = ["sofifa_id", "long_name", "international_reputation", "weak_foot", "skill_moves", "work_rate", "body_type", "real_face", "release_clause_eur", "player_tags", "team_position", "team_jersey_number", "loaned_from",
"joined", "contract_valid_until", "nation_position", "nation_jersey_number", "pace", "shooting", "passing", "dribbling", "defending", "physic",
"gk_diving", "gk_handling", "gk_kicking", "gk_reflexes", "gk_speed", "gk_positioning", "player_traits", "attacking_crossing", "attacking_finishing",
"attacking_heading_accuracy", "attacking_short_passing", "attacking_volleys", "skill_dribbling", "skill_curve", "skill_fk_accuracy", "skill_long_passing", "skill_ball_control",
"movement_acceleration", "movement_sprint_speed", "movement_agility", "movement_reactions", "movement_balance", "power_shot_power", "power_jumping", "power_stamina",
"power_strength", "power_long_shots", "mentality_aggression", "mentality_interceptions", "mentality_positioning", "mentality_vision", "mentality_penalties", "mentality_composure",
"defending_marking", "defending_standing_tackle", "defending_sliding_tackle", "goalkeeping_diving", "goalkeeping_handling", "goalkeeping_kicking", "goalkeeping_positioning", "goalkeeping_reflexes"]

player_skill_transformed= players_df[player_skill_cols].copy()

player_skill_transformed.head()

,sofifa_id,long_name,international_reputation,weak_foot,skill_moves,work_rate,body_type,real_face,release_clause_eur,player_tags,...,mentality_penalties,mentality_composure,defending_marking,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes
0,158023,Lionel Andrés Messi Cuccittini,5,4,4,Medium/Low,Messi,Yes,195800000.0,"#Dribbler, #Distance Shooter, #Crosser, #FK Sp...",...,75,96,33,37,26,6,11,15,14,8
1,20801,Cristiano Ronaldo dos Santos Aveiro,5,4,5,High/Low,C. Ronaldo,Yes,96500000.0,"#Speedster, #Dribbler, #Distance Shooter, #Acr...",...,85,95,28,32,24,7,11,15,14,11
2,190871,Neymar da Silva Santos Junior,5,5,5,High/Medium,Neymar,Yes,195200000.0,"#Speedster, #Dribbler, #Playmaker , #Crosser,...",...,90,94,27,26,29,9,9,15,15,11
3,200389,Jan Oblak,3,3,1,Medium/Medium,Normal,Yes,164700000.0,NaN,...,11,68,27,12,18,87,92,78,90,89
4,183277,Eden Hazard,4,4,4,High/Medium,Normal,Yes,184500000.0,"#Speedster, #Dribbler, #Acrobat",...,88,91,34,27,22,11,12,6,8,8


In [11]:
### Capture unique countries for Country table from players dataset

unique_countries = players_transformed.nationality.unique()

### Sort the countries in alphabetical order
unique_countries_sorted = np.sort(unique_countries)

### Create dataframe for countries
unique_countries_df = pd.DataFrame(unique_countries_sorted, columns=['country'])
unique_countries_df.reset_index(inplace=True)
unique_countries_df.rename(columns={"nationality": "country_name", "index": "country_id"}, inplace=True)

unique_countries_df.head()

,country_id,country
0,0,Afghanistan
1,1,Albania
2,2,Algeria
3,3,Angola
4,4,Antigua & Barbuda


### Scrape list of Team URLs from league listing tables in FBRef.com

In [12]:
# print('Enter Web URL to Scrape Team listing (e.g. https://fbref.com/en/comps/9/Premier-League-Stats)')
# league_url = input()

league_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [13]:
# Create Connection to League URL and read Beautiful soup
r = requests.get(league_url)
soup = BeautifulSoup(r.text, "html.parser")

In [14]:
# Create empty list to append Team urls into
team_urls = []

# Scrape for items that have the attribute "data-stat": "squad" and save as teams
teams = soup.find_all(attrs={"data-stat": "squad"})

# Iterate through all items in teams to find the <a> that contains links
for team in teams:
    x = team.find('a')
    # Use following if statement to weed out any <a> tags that do not have href so we dont get an 'href' error
    if x is not None and 'href' in x.attrs:
        # Append all hrefs onto the team_urls
        team_urls.append(x['href'])


In [15]:
# Team URLs appear twice on the team_urls
# Use the following to get a final list where the team urls only appear once
half = int(len(team_urls)/2)
team_urls_half = team_urls[0:int(half)]

# Add the begining of the URL
team_list = []
for x in team_urls_half:
     team_list.append("https://fbref.com" + x)
        
# team_list

In [17]:
#Read team table direcly into dataframe
d_list = pd.DataFrame()
# Place holder to delete last row "Squad Total" before appending to d_list
place_holder = pd.DataFrame()

for team in team_list:
    dfs = pd.read_html(team,header=1)
    place_holder = dfs[0]
    place_holder.drop(place_holder.tail(1).index,inplace=True)
    d_list = d_list.append(place_holder)
d_list.head(100)


,Player,Nation,Pos,Age,MP,Starts,Min,Gls,Ast,PK,...,G+A-PK,xG,npxG,xA,xG.1,xA.1,xG+xA,npxG.1,npxG+xA,Matches
0,Virgil van Dijk,nl NED,DF,28.0,24,24,2160.0,4.0,0.0,0.0,...,0.17,1.4,1.4,0.5,0.06,0.02,0.08,0.06,0.08,Matches
1,Trent Alexander-Arnold,eng ENG,DF,20.0,24,23,2100.0,2.0,10.0,0.0,...,0.51,1.8,1.8,7.3,0.08,0.31,0.39,0.08,0.39,Matches
2,Andrew Robertson,sco SCO,DF,25.0,24,23,2085.0,1.0,6.0,0.0,...,0.30,1.1,1.1,4.9,0.05,0.21,0.26,0.05,0.26,Matches
3,Georginio Wijnaldum,nl NED,MF,28.0,23,23,1950.0,2.0,0.0,0.0,...,0.09,1.5,1.5,1.0,0.07,0.05,0.11,0.07,0.11,Matches
4,Roberto Firmino,br BRA,FW,27.0,24,22,1983.0,8.0,4.0,0.0,...,0.54,9.3,9.3,4.2,0.42,0.19,0.61,0.42,0.61,Matches
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22,Billy Gilmour,sco SCO,"MF,FW",18.0,2,0,9.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.1,0.00,0.57,0.57,0.00,0.57,Matches
23,Willy Caballero,ar ARG,GK,37.0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Matches
24,Marc Guéhi,eng ENG,DF,19.0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Matches
25,Kenedy,br BRA,"FW,MF",23.0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Matches


In [18]:
#Rename columns to remove special characters and better formatting

d_list.rename(columns={'Player': 'long_name', 'Nation': 'nationality', 'Pos': 'team_position', 'Age': 'age', 'MP': 'ptime_matches_played',
                  'Starts': 'ptime_starts', 'Min': 'ptime_min', 'Gls': 'perf_goals', 'Ast': 'perf_assists', 'PK': 'perf_pks',
                  'PKatt': 'perf_pkatt', 'CrdY': 'perf_crd_y', 'CrdR': 'perf_crd_r', 'Gls.1': 'per_90_goals', 'Ast.1': 'per_90_assts',
                  'G+A': 'per_90_ga', 'G-PK': 'per_90_g_pk', 'G+A-PK': 'per_90_g_a_pk', 'xG': 'exp_goals', 'npxG': 'exp_np_exp_goals',
                  'xA': 'exp_assists', 'xG.1': 'per_90_exp_goals', 'xA.1': 'per_90_exp_assists', 'xG+xA': 'per_90_exp_goals_exp_assists',
                   'npxG.1': 'per_90_exp_np_exp_goals', 'npxG+xA': 'per_90_exp_np_exp_goals_assists'}, inplace=True)

In [19]:
df_player_perf = d_list[{'long_name','ptime_matches_played','ptime_starts', 'ptime_min', 'perf_goals', 'perf_assists', 'perf_pks', 'perf_pkatt',
 'perf_crd_y', 'perf_crd_r', 'per_90_goals', 'per_90_assts', 'per_90_ga', 'per_90_g_pk', 'per_90_g_a_pk', 'exp_goals', 'exp_np_exp_goals', 'exp_assists',
                      'per_90_exp_goals', 'per_90_exp_assists', 'per_90_exp_goals_exp_assists', 'per_90_exp_np_exp_goals', 'per_90_exp_np_exp_goals_assists'}]
df_player_perf.head()

,long_name,perf_goals,per_90_exp_goals_exp_assists,exp_goals,ptime_min,per_90_assts,exp_np_exp_goals,exp_assists,perf_pkatt,per_90_ga,...,per_90_goals,perf_pks,ptime_starts,per_90_exp_np_exp_goals,per_90_g_pk,perf_crd_r,perf_assists,per_90_exp_assists,ptime_matches_played,per_90_exp_goals
0,Virgil van Dijk,4.0,0.08,1.4,2160.0,0.00,1.4,0.5,0.0,0.17,...,0.17,0.0,24,0.06,0.17,0.0,0.0,0.02,24,0.06
1,Trent Alexander-Arnold,2.0,0.39,1.8,2100.0,0.43,1.8,7.3,0.0,0.51,...,0.09,0.0,23,0.08,0.09,0.0,10.0,0.31,24,0.08
2,Andrew Robertson,1.0,0.26,1.1,2085.0,0.26,1.1,4.9,0.0,0.30,...,0.04,0.0,23,0.05,0.04,0.0,6.0,0.21,24,0.05
3,Georginio Wijnaldum,2.0,0.11,1.5,1950.0,0.00,1.5,1.0,0.0,0.09,...,0.09,0.0,23,0.07,0.09,0.0,0.0,0.05,23,0.07
4,Roberto Firmino,8.0,0.61,9.3,1983.0,0.18,9.3,4.2,0.0,0.54,...,0.36,0.0,22,0.42,0.36,0.0,4.0,0.19,24,0.42


### Fuzzy Match 

In [26]:
# Extract sofifa_id and long_name from players_transfored 
sofifa_id = players_transformed[['sofifa_id', 'long_name']]

# Extract long_name from df_player_perf
fbref_names = df_player_perf[['long_name']]


In [27]:
# First get all exact matches using left merge because we need only find the IDs for fbref_names
merged_df = fbref_names.merge(sofifa_id, 
                        left_on='long_name', 
                        right_on='long_name', 
                        how='left', 
                        suffixes=["_sofifa","_fbref"] #since they are the same name, we need a suffix
                       )


In [28]:
# Make to subsets one with players  missing sofifaID and one with IDs
missing_ids = merged_df[merged_df.sofifa_id.isnull()]
matched_ids = merged_df.dropna()

# How many players are missing an id
print("Number of players missing IDs:" + str(len(missing_ids)))
missing_ids

Number of players missing IDs:170


,long_name,sofifa_id
4,Roberto Firmino,NaN
5,Mohamed Salah,NaN
7,Jordan Henderson,NaN
8,Alisson,NaN
9,Fabinho,NaN
...,...,...
527,Max Aarons,NaN
530,Emi Buendía,NaN
533,Alexander Tettey,NaN
541,Onel Hernández,NaN


In [30]:
# Create match_name using fuzz from fuzzywuzzy package

def match_name(name, list_names, min_score=0):
    # -1 score incase we don't get any matches
    max_score = -1
    # Returning empty name for no match as well
    max_name = ""
    # Iternating over all names in the other
    for name2 in list_names:
        #Finding fuzzy match score
        score = fuzz.ratio(name, name2)
        # Checking if we are above our threshold and have a better score
        if (score > min_score) & (score > max_score):
            max_name = name2
            max_score = score
    return (max_name, max_score)

In [33]:
# List dicts of scores from match_name for easy reading
dict_list = []

# iterating over our players without IDs found above
for name in missing_ids['long_name']:
    # Use our method to find best match, we can set a threshold here
    match = match_name(name, sofifa_id['long_name'], 75)
    
    # New dict for storing data
    dict_ = {}
    dict_.update({"player_name" : name})
    dict_.update({"match_name" : match[0]})
    dict_.update({"score" : match[1]})
    dict_list.append(dict_)
    
results_table = pd.DataFrame(dict_list)
# Display results
results_table

,player_name,match_name,score
0,Roberto Firmino,,-1
1,Mohamed Salah,Mohamed Sylla,85
2,Jordan Henderson,Jordan Brian Henderson,84
3,Alisson,,-1
4,Fabinho,,-1
...,...,...,...
165,Max Aarons,,-1
166,Emi Buendía,Emiliano Buendía,81
167,Alexander Tettey,Alexander Banor Tettey,84
168,Onel Hernández,Daniel Hernández,87


In [34]:
# Table with names with a score higher than 75
score_above75 = results_table[(results_table.score > 75)]
print("There are " + str(len(score_above75)) + " matches with a score above 75.")

# Table with names with no match
score_0 = results_table[(results_table.score < 0)]


There are 90 matches with a score above 75.


In [55]:
# Merge our inital dataset of missing_ids with the newly matched datatset that has a score over 75 from fuzzy match
missing_ids_merged = missing_ids.merge(score_above75,
                                       left_on = 'long_name',
                                       right_on = "player_name",
                                       how='left',
                                       suffixes=["_fbref","_match"] 
                                      )
# Drop unnecessary colummns, we only need the 'long_name', "match_name"=sofifa's long name
# Also drop rows with N/A under "player_name" we will not be able to get sofifa ids for those
# Make sure you have the right amount of names left (in this case should be 90)

missing_ids_merged = missing_ids_merged[['long_name','match_name']]
missing_ids_merged = missing_ids_merged.dropna()


In [36]:
# Now we can finally match the remaing fbref name to the sofifa ids
missing_ids_matched = missing_ids_merged.merge(sofifa_id,
                                       left_on = 'match_name',
                                       right_on = "long_name",
                                       how='left',
                                       suffixes=["_fbref","_sofifa"] 
                                      )
# Keep only needed columns, long_name_fbref and sofifa_id and rename so it matched with the dataframe we will me appending to
missing_ids_matched = missing_ids_matched[['long_name_fbref', 'sofifa_id']]
missing_ids_matched = missing_ids_matched.rename(columns = {"long_name_fbref" : "long_name"})

missing_ids_matched.head()


,long_name,sofifa_id
0,Mohamed Salah,246872
1,Jordan Henderson,183711
2,James Milner,138412
3,Adam Lallana,180819
4,Andy Lonergan,118794


In [38]:
# Add the newly matched ids to the extact matched ids
fully_matched_ids = matched_ids.append(missing_ids_matched)

# Check how many names we were able to match ids to 
print("We had a dataset of " + str(len(fbref_names)) + " names from fbref, we were able to match " + 
     str(len(fully_matched_ids)) + " of them.")

We had a dataset of 549 names from fbref, we were able to match 476 of them.


In [43]:
df_player_perf = df_player_perf.merge(fully_matched_ids,
                                      left_on ='long_name',
                                      right_on = 'long_name',
                                      how = 'left',
                                      suffixes=["_fbref","_match"] 
                                      )

In [54]:
df_player_perf['sofifa_id'].isna().sum() 

80

# Load

### Create DB Connection

In [56]:
#modify connection string using local database Id and Password 
#connection_string = "postgres:{Password}@localhost:5432/fifa_db"
connection_string = "postgres:postgres@localhost:5432/fifa_db"

engine = create_engine(f'postgresql://{connection_string}')

### Load DataFrames into SQL

In [57]:
#Import Player information dataframes into SQL tables, players, player_skill, country
players_transformed.to_sql(name='players', con=engine, if_exists='replace', index=False)
player_skill_transformed.to_sql(name='player_skill', con=engine, if_exists='replace', index=False)
df_player_perf.to_sql(name='player_perf', con=engine, if_exists='replace', index=False)
unique_countries_df.to_sql(name='country', con=engine, if_exists='replace', index=False)


In [58]:
#confirm players table
pd.read_sql_query('select * from players', con=engine).head()

,sofifa_id,player_url,short_name,long_name,age,dob,height_cm,weight_kg,nationality,club,overall,potential,value_eur,wage_eur,player_positions,preferred_foot
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,32,1987-06-24,170,72,Argentina,FC Barcelona,94,94,95500000,565000,"RW, CF, ST",Left
1,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,34,1985-02-05,187,83,Portugal,Juventus,93,93,58500000,405000,"ST, LW",Right
2,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Junior,27,1992-02-05,175,68,Brazil,Paris Saint-Germain,92,92,105500000,290000,"LW, CAM",Right
3,200389,https://sofifa.com/player/200389/jan-oblak/20/...,J. Oblak,Jan Oblak,26,1993-01-07,188,87,Slovenia,Atlético Madrid,91,93,77500000,125000,GK,Right
4,183277,https://sofifa.com/player/183277/eden-hazard/2...,E. Hazard,Eden Hazard,28,1991-01-07,175,74,Belgium,Real Madrid,91,91,90000000,470000,"LW, CF",Right


In [59]:
#confirm player_skill table
pd.read_sql_query('select * from player_skill', con=engine).head()

,sofifa_id,long_name,international_reputation,weak_foot,skill_moves,work_rate,body_type,real_face,release_clause_eur,player_tags,...,mentality_penalties,mentality_composure,defending_marking,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes
0,158023,Lionel Andrés Messi Cuccittini,5,4,4,Medium/Low,Messi,Yes,195800000.0,"#Dribbler, #Distance Shooter, #Crosser, #FK Sp...",...,75,96,33,37,26,6,11,15,14,8
1,20801,Cristiano Ronaldo dos Santos Aveiro,5,4,5,High/Low,C. Ronaldo,Yes,96500000.0,"#Speedster, #Dribbler, #Distance Shooter, #Acr...",...,85,95,28,32,24,7,11,15,14,11
2,190871,Neymar da Silva Santos Junior,5,5,5,High/Medium,Neymar,Yes,195200000.0,"#Speedster, #Dribbler, #Playmaker , #Crosser,...",...,90,94,27,26,29,9,9,15,15,11
3,200389,Jan Oblak,3,3,1,Medium/Medium,Normal,Yes,164700000.0,None,...,11,68,27,12,18,87,92,78,90,89
4,183277,Eden Hazard,4,4,4,High/Medium,Normal,Yes,184500000.0,"#Speedster, #Dribbler, #Acrobat",...,88,91,34,27,22,11,12,6,8,8


In [60]:
#confirm country table
pd.read_sql_query('select * from player_perf', con=engine).head()

,long_name,perf_goals,per_90_exp_goals_exp_assists,exp_goals,ptime_min,per_90_assts,exp_np_exp_goals,exp_assists,perf_pkatt,per_90_ga,...,perf_pks,ptime_starts,per_90_exp_np_exp_goals,per_90_g_pk,perf_crd_r,perf_assists,per_90_exp_assists,ptime_matches_played,per_90_exp_goals,sofifa_id
0,Virgil van Dijk,4.0,0.08,1.4,2160.0,0.00,1.4,0.5,0.0,0.17,...,0.0,24,0.06,0.17,0.0,0.0,0.02,24,0.06,203376.0
1,Trent Alexander-Arnold,2.0,0.39,1.8,2100.0,0.43,1.8,7.3,0.0,0.51,...,0.0,23,0.08,0.09,0.0,10.0,0.31,24,0.08,231281.0
2,Andrew Robertson,1.0,0.26,1.1,2085.0,0.26,1.1,4.9,0.0,0.30,...,0.0,23,0.05,0.04,0.0,6.0,0.21,24,0.05,216267.0
3,Georginio Wijnaldum,2.0,0.11,1.5,1950.0,0.00,1.5,1.0,0.0,0.09,...,0.0,23,0.07,0.09,0.0,0.0,0.05,23,0.07,181291.0
4,Roberto Firmino,8.0,0.61,9.3,1983.0,0.18,9.3,4.2,0.0,0.54,...,0.0,22,0.42,0.36,0.0,4.0,0.19,24,0.42,NaN
